In [1]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 27.7 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.9/807.9 kB 13.2 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 11.9 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.0/26.0 MB 43.7 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 38.5 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.2 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 8.9 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-xmnso5x7/overlay/local/bin
    changing mode of /tmp/pip-build-env-xmnso5x7/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-xmnso5x7/overlay/

**Importing the data**

In [5]:
import pandas as pd

df = pd.read_csv('/content/Lisensing_2010.csv')
df.head()

,Unnamed: 0,Document_id,Content
0,0,Document NRG0000020100611e66b0002b,Companies Selected For Next Stage In UK Dounre...
1,1,Document FF00000020100618e66i00009,Thermogenesis Files 8K - Entry Into Definitive...
2,2,Document DJI0000020100615e66f000bp,Int. Brand Licensing Acquisition And Placing...
3,3,Document BWR0000020100615e66f00192,"Greyson Hosts Conference Call on Wednesday, Ju..."
4,4,Document BWR0000020100616e66g00445,JAKKS Pacific’s CDI Division Recognized with T...


**Taking the subset of the data**

In [6]:
data = []
data.append(df.iloc[0][2])


In [7]:

model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

**Importing all the Libraries**

In [3]:
from huggingface_hub import hf_hub_download


In [4]:
from llama_cpp import Llama

**Model Download**

In [8]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

llama-2-13b-chat.ggmlv3.q5_1.bin:   0%|          | 0.00/9.76G [00:00<?, ?B/s]

**Loading the model**

In [9]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=1024, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


**Initialize the GPU**

In [10]:
# See the number of layers in GPU
lcpp_llm.params.n_gpu_layers

32

**Split the License data into chunks**

In [ ]:
# Splitting the conversation into chunks
chunks = [data[0][i:i+512] for i in range(0, len(data[0]), 512)]

# Iterate through the chunks and generate responses
for chunk in chunks:
    prompt_template = f'''SYSTEM: You are a helpful, respectful, and honest assistant. Always answer as helpfully.

USER: Identify technology from the below paragraph.
{chunk}

ASSISTANT:
'''
    response = lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.3, top_p=0.95,
                        repeat_penalty=1.2, top_k=150,
                        echo=True)
    print(response)


In [ ]:
print(response["choices"][0]["text"])

In [ ]:
print(data)

**Response for the Entire 2010 CSV file**

In [25]:
def get_reponse_from_chuks(data):
  # Splitting the conversation into chunks
  chunks = [data[i:i+512] for i in range(0, len(data), 512)]

  # Iterate through the chunks and generate responses
  for chunk in chunks:
      prompt_template = f'''SYSTEM: You are a helpful, respectful, and honest assistant. Always answer as helpfully.

  USER: Identify any technology used in the below license document.
  {chunk}

  ASSISTANT:
  '''
      response = lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.3, top_p=0.95,
                          repeat_penalty=1.2, top_k=150,
                          echo=True)
      return response


In [ ]:
import pandas as pd
import os
import csv

# Assuming you have a DataFrame called 'df' containing your data
contents = df['Content'][0:50]  # Select rows 200 to 300

# Define a directory to save the CSV files
output_dir = 'output_csv'
os.makedirs(output_dir, exist_ok=True)

# Create a single CSV file to store all responses
output_file = os.path.join(output_dir, 'responses.csv')

with open(output_file, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)

    for content in contents:
        response = get_reponse_from_chuks(content)
        response_text = response["choices"][0]["text"]
        assistant_response = response_text.split("ASSISTANT:")[1].strip()
        assistant_response = assistant_response.replace('\n', ' ')

        print(assistant_response)
        csv_writer.writerow([assistant_response])


Llama.generate: prefix-match hit


1. The technology used in this license document is likely to be Microsoft Word or another word processing software, as it appears to be a typed document with formatting and copyright information at the bottom.   2. There are no explicit mentions of any specific technologies being used at the Dounreay nuclear research facility itself, but it can be inferred that various nuclear technology and equipment may be employed there given its purpose as a research facility for nuclear energy.


Llama.generate: prefix-match hit


Based on the provided license document, there is no mention of any specific technology used. The agreement appears to be focused on providing assurance to CBR Systems, Inc. regarding ThermoGenesis Corp.'s continuing efforts and commitment to the licensing arrangement. It does not appear that any particular technology is being referred to or utilized in this document. Is there anything else I can help with?


Llama.generate: prefix-match hit


...

Llama.generate: prefix-match hit


Based on the information provided, there is one piece of technology mentioned in the license document:    1. Lung sound analysis technology.


Llama.generate: prefix-match hit


1. The technology used in this document is likely to be Microsoft Word, as it appears to be a press release written in English with proper formatting and includes company names, dates, and locations. Additionally, the use of copyright symbols (©) suggests that the document may have been created using word processing software such as Microsoft Word.   2. The license agreement is likely to be managed through an online platform or database, which would allow for easy access and tracking of licensing agreements. This could include a cloud-based storage solution like Dropbox or Google Drive, or a specialized licensing management system like CPA Global or MarkMonitor.   3. The document may also have been reviewed using grammar and spell check software to ensure accuracy and professionalism in the language used.


Llama.generate: prefix-match hit


In [ ]:
# Access the 3rd column of the DataFrame
contents = df['Content']

for content in contents:
  response = get_reponse_from_chuks(content)
  print("===========================================================")
  print(response["choices"][0]["text"]+"\n")
  print("============================================================")


In [ ]:
# Iterate through the contents
for content in contents:
    response = get_response_from_chunks(content)

    # Append the response to the DataFrame
    responses_df = responses_df.append({"Response": response}, ignore_index=True)

# Save the DataFrame to a CSV file
responses_df.to_csv("responses.csv", index=False)